This file will learn follow reference https://towardsdatascience.com/named-entity-recognition-ner-meeting-industrys-requirement-by-applying-state-of-the-art-deep-698d2b3b4ede

In [1]:
def read_all_line( directory , file ):
    file = open( directory + "/" + file , "r" , encoding="utf-8" )
    contents = file.read()
    return contents

In [2]:
def individual_preprocess( raw_data , split_sentence , count ,remove = [] ):
    pre_data_frame = { "Sentence" : [] , "Word" : [] , "Tag" : [] }
    for word in raw_data:
        if word in remove :
            None
        elif word in split_sentence :
            count += 1
        else: 
            check = word.find('(')
            if check > 0:
                pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word[ 0 : word.find('(')] )
                pre_data_frame["Tag"].append( word[ word.find('(') + 1 : word.find(')')] )
            else:
                pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word )
                pre_data_frame["Tag"].append( "O" )
    return pre_data_frame , count

In [3]:
import pandas as pd
def preprocess( directory , list_file , count = 1 , individual = False ):
    data_frame = pd.DataFrame( { "Sentence" : [] , "Word" : [] , "Tag" : [] } )
    if( individual ):
        word = read_all_line( directory , list_file ).split('|')
        pre_data_frame , count = individual_preprocess( word ,
                                              ["\n"] ,
                                              count ,
                                              [ "" , '' , '\0'] )
        data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    else:
        for file in list_file :
            word = read_all_line( directory , file ).split('|')
            pre_data_frame , count = individual_preprocess( word ,
                                              ["\n"] ,
                                              count ,
                                              [ "" , '' , '\0'] )
            data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    return data_frame

In [4]:
#directory of corpus
corpus_directory = "corpus_directory"
corpus_file = ("Politic9.txt" , "Allcolumn.txt")
corpus_files = ("Politic9.txt" , "Allcolumn.txt" , "98JUL5_1.txt" ,
                "98JUL5_2.txt" , "bkknews1.txt" , "Result1.txt" ,
                "Result2(corpus-1).txt" , "Result3(corpus-2).txt" ,
                "result4.txt" )
corpus_test = ("Result_c_2_1.txt", "Result_c_2_2.txt")
all_corpus = corpus_files + corpus_test
individual = False

In [5]:
data_frame = preprocess( corpus_directory , corpus_files )
data_frame
number_words = len( set( list( data_frame["Word"].values ) ) )
number_tags = len( list( set( data_frame["Tag"].values ) ) )
print( "Have type of tag is " + str( number_tags ) + 
        " tags from " + str(number_words) + " words")
# print(set( list( data_frame["Tag"].values )))

Have type of tag is 14 tags from 6966 words


In [6]:
data_frame

,Sentence,Word,Tag
0,Sentence: 1,"""",O
1,Sentence: 1,ทักษิณ,per
2,Sentence: 1,"""",O
3,Sentence: 1,,O
4,Sentence: 1,ขอ,O
...,...,...,...
67632,Sentence: 4010,แสดง,O
67633,Sentence: 4010,ความ,O
67634,Sentence: 4010,รู้สึก,O
67635,Sentence: 4010,ในทำนองเดียวกัน,O


Follow https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2

In [7]:
data_frame['Sentence'].nunique(), data_frame.Word.nunique(), data_frame.Tag.nunique()

(4010, 6966, 14)

In [8]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [9]:
data_frame = data_frame.fillna(method='ffill')
print( "Number of sentence is {:d}".format(data_frame['Sentence'].nunique()) )
print( "Number of word is {:d}".format( data_frame.Word.nunique() ) )
print( "Number of tag is {:d}".format( data_frame.Tag.nunique() ) )

Number of sentence is 4010
Number of word is 6966
Number of tag is 14


In [10]:
data_frame.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,,21
1,O,56284
2,loc,2077
3,loc_cont,39
4,loc_end,120
5,loc_start,120
6,org,2242
7,org_cont,1721
8,org_end,1020
9,org_start,1032


In [11]:
data_train = data_frame.drop('Tag', axis=1)
#vector_handle = DictVectorizer(sparse=False)
#data_train = vector_handle.fit_transform( data_train.to_dict('records') )
#value_train = data_frame.Tag.values

In [12]:
test_frame = preprocess( corpus_directory , corpus_test , 1 , individual )
#data_test = test_frame.drop('Tag', axis=1)
#vector_handle = DictVectorizer(sparse=False)
#data_test = vector_handle.fit_transform( data_test.to_dict('records') )
#value_test = test_frame.Tag.values

In [13]:
data_frame.shape[0] , test_frame.shape[0]

(67637, 35746)

In [14]:
all_frame = data_frame.append( test_frame )
all_data = all_frame.drop('Tag', axis=1)
vector_handle = DictVectorizer(sparse=False)
all_data = vector_handle.fit_transform(all_data.to_dict('records'))
all_value = all_frame.Tag.values
size_of_test = 1.0 * test_frame.shape[0] / all_data.shape[0]
print( "Propotional of test is " + str( size_of_test ) )

classes = np.unique( all_value )
classes = classes.tolist()
new_classes = classes.copy()
new_classes.remove('O')
new_classes

Propotional of test is 0.34576284302061266


['',
 'loc',
 'loc_cont',
 'loc_end',
 'loc_start',
 'org',
 'org_cont',
 'org_end',
 'org_start',
 'per',
 'per_cont',
 'per_end',
 'per_start']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(all_data, 
        all_value, 
        test_size = size_of_test, 
        shuffle = False )

In [16]:
# Use Perceptron
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


-- Epoch 1-- Epoch 1
-- Epoch 1
-- Epoch 1
-- Epoch 1

-- Epoch 1-- Epoch 1

-- Epoch 1
Norm: 7.75, NNZs: 54, Bias: -4.000000, T: 67637, Avg. loss: 0.001257
Total training time: 10.95 seconds.
-- Epoch 1
Norm: 68.96, NNZs: 2618, Bias: -2.000000, T: 67637, Avg. loss: 0.024025
Total training time: 11.02 seconds.
Norm: 15.30, NNZs: 214, Bias: -3.000000, T: 67637, Avg. loss: 0.002972
Total training time: 11.05 seconds.
-- Epoch 1
Norm: 14.49, NNZs: 187, Bias: -2.000000, T: 67637, Avg. loss: 0.002543
Total training time: 11.06 seconds.
-- Epoch 1-- Epoch 1

Norm: 50.58, NNZs: 1550, Bias: -4.000000, T: 67637, Avg. loss: 0.049233
Total training time: 11.14 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   3 out of  14 | elapsed:   11.2s remaining:   41.1s
[Parallel(n_jobs=-1)]: Done   5 out of  14 | elapsed:   11.3s remaining:   20.3s


Norm: 56.76, NNZs: 1737, Bias: -2.000000, T: 67637, Avg. loss: 0.017978
Total training time: 11.37 seconds.
-- Epoch 1
Norm: 7.07, NNZs: 41, Bias: -4.000000, T: 67637, Avg. loss: 0.000488
Total training time: 11.42 seconds.
Norm: 129.34, NNZs: 7097, Bias: 1.000000, T: 67637, Avg. loss: 0.154013
Total training time: 11.57 seconds.


[Parallel(n_jobs=-1)]: Done   7 out of  14 | elapsed:   11.6s remaining:   11.6s


Norm: 36.96, NNZs: 1018, Bias: -4.000000, T: 67637, Avg. loss: 0.009255Norm: 46.67, NNZs: 1534, Bias: -4.000000, T: 67637, Avg. loss: 0.024321
Total training time: 3.42 seconds.

Total training time: 3.32 seconds.
Norm: 45.52, NNZs: 1575, Bias: -1.000000, T: 67637, Avg. loss: 0.016012
Total training time: 3.09 seconds.
Norm: 46.56, NNZs: 1470, Bias: -2.000000, T: 67637, Avg. loss: 0.026568
Total training time: 3.45 seconds.
Norm: 32.65, NNZs: 849, Bias: -5.000000, T: 67637, Avg. loss: 0.028416Norm: 46.24, NNZs: 1629, Bias: -2.000000, T: 67637, Avg. loss: 0.015125
Total training time: 3.40 seconds.
Total training time: 3.35 seconds.



[Parallel(n_jobs=-1)]: Done   9 out of  14 | elapsed:   14.6s remaining:    8.1s
[Parallel(n_jobs=-1)]: Done  11 out of  14 | elapsed:   14.6s remaining:    4.0s
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:   14.7s finished


Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=5, n_iter_no_change=5, n_jobs=-1,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=10, warm_start=False)

In [17]:
print(classification_report(y_pred=per.predict(X_test), 
                            y_true=y_test, labels=new_classes))

/home/sevinar/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

                   0.00      0.00      0.00        12
         loc       0.78      0.58      0.67       912
    loc_cont       0.00      0.00      0.00        52
     loc_end       0.09      0.02      0.03        63
   loc_start       0.45      0.40      0.42        63
         org       0.79      0.22      0.34       953
    org_cont       0.42      0.04      0.07       749
     org_end       0.59      0.05      0.08       509
   org_start       0.38      0.14      0.20       521
         per       0.80      0.01      0.03       299
    per_cont       0.00      0.00      0.00       456
     per_end       0.55      0.06      0.11       406
   per_start       0.21      0.16      0.18       406

   micro avg       0.59      0.18      0.28      5401
   macro avg       0.39      0.13      0.16      5401
weighted avg       0.53      0.18      0.24      5401



In [18]:
# Use Passive Aggressive Classifier
pa =PassiveAggressiveClassifier()
pa.partial_fit(X_train, y_train, classes)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=1000, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [ ]:
print(classification_report(y_pred=pa.predict(X_test),
                            y_true=y_test, labels=new_classes))

In [ ]:
# use naive bayes # NOT WORK
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

In [ ]:
print(classification_report(y_pred=nb.predict(X_test), 
                            y_true=y_test, labels = new_classes))